# **Download Large SQL Queries in Python**
* When using the Databricks Cloud UI to download the results of a SQL query, the results are limited to the first 1000 rows.
* This example contains code to help you access the entire dataset.
* First, the notebook walks through an explanation how to do this.
* At the end, we give a simple function you can run in your notebooks to do this.
* I have modified and added some code?udfs defined and used?to make it more interesting -- Jules

### **Setup**
Let's create a SQL table with more than 1000 rows.

In [3]:
from pyspark.sql import Row
##square the numbers
def squared (s):
  return s * s
#
## cube the number
#
def cubed (s):
  return s * s * s
#
## register them as UDF so we can use them in SQL
#Row(key="b", group="consonants", value=2),
sqlContext.udf.register("squaredWithPython", squared)
sqlContext.udf.register("cubedWithPython", cubed)
##now create some rows 
array = []
for i in range(0, 5000):
  array.append(Row(num=i, square=squared(i), cube=cubed(i)))
#create a Dataframe
dataFrame = sqlContext.createDataFrame(sc.parallelize(array))
#register the Dataframe as temporary table to which we can issue some SQL queries.
#
dataFrame.registerTempTable("test_download_table")
## use UDF now for another table
sqlContext.range(1, 20).registerTempTable("test_udfs")

In [4]:
%sql 
-- Clicking on the download button on the bottom right for this table will download only the first 1000 rows.
select * from test_download_table

Use UDFS defined above with test_udf tables

In [6]:
%sql describe test_udfs

In [7]:
%sql select * from test_udfs

In [8]:
%sql select id, squaredWithPython(id) as id_squared from test_udfs
-- use the UDF defined above to square numbers in the test_udfs table

In [9]:
%sql select id, cubedWithPython(id) as id_cubed from test_udfs
--use the udf defined above to cube the numbers in the test_udfs table

### Solution 1: Make the output results available for all Databricks users.

If you only need to allow Databricks users to read these output files, then you can save the file to our [FileStore]("group_id") and download it there.

In [11]:
sql_query = "select * from test_download_table"
dbutils.fs.rm("dbfs:/FileStore/test_download_table", True)
sqlContext.sql(sql_query).coalesce(1).write.format("com.databricks.spark.csv").save("dbfs:/FileStore/test_download_table")

You can access a file in the File store by going to [files/test_download_table/part-00000](files/test_download_table/part-00000)

### Solution 2: Make the output results universally accessible.

If you want the files to be accessible by non-Databricks users, following these steps:

#### Step 1: Configure your results to be saved in an S3 bucket.

In [15]:
ACCESS_KEY = "YOUR_ACCESS_KEY"
SECRET_KEY = "YOUR_SECRET_KEY"
AWS_BUCKET_NAME = "YOUR_AWS_BUCKET_NAME"
FOLDER_FOR_RESULTS = "YOUR_DESTINATION_FOLDER"

#### Step 2: Save the results to an S3 bucket.

The **saveAsTextFile** command is safe even on a very large RDD as each worker writes out it's partitions directly.

In [17]:
sql_query = "select * from test_download_table"
sqlContext.sql(sql_query).coalesce(1).write.format("com.databricks.spark.csv").save("s3n://%s:%s@%s/%s" % (ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME, FOLDER_FOR_RESULTS))

#### Step 3: Call the S3 library to generate links for downloading these files.

Set the desired timeout you would like the links to be valid for.

In [19]:
from boto.s3.connection import S3Connection
conn = S3Connection(ACCESS_KEY, SECRET_KEY)
bucket = conn.get_bucket(AWS_BUCKET_NAME)
# NOTE: There are as many output files as there were partitions in the RDD.
# coalesce() or repartition() can be called on RDD to explicity set the desired number of output files.
rs = bucket.list("%s/part" % FOLDER_FOR_RESULTS)

html_output = ""
for key in rs:
  html_output += "<a href='%s'>Download %s</a><br/>" % (key.generate_url(expires_in=1440, query_auth=True), key.name)

#### Step 4: Use displayHTML to show the clickable download links.

In [21]:
displayHTML(html_output)

#### **Final: Put this together into a function that can be called to create the displayed links.**
Use the **output_links_to_download_sql** function from now on rather than all the steps above.

In [23]:
def output_links_to_download_sql(sql_query, num_output_files, accesskey, secretkey, aws_bucket_name, aws_folder):
  sqlContext.sql(sql_query).coalesce(num_output_files).write.format("com.databricks.spark.csv").save("s3n://%s:%s@%s/%s" % (accesskey, secretkey, aws_bucket_name, aws_folder))
  from boto.s3.connection import S3Connection
  conn = S3Connection(accesskey, secretkey)
  bucket = conn.get_bucket(aws_bucket_name)
  rs = bucket.list("%s/part" % aws_folder)
  html_output = ""
  for key in rs:
    html_output += "<a href='%s'>Download %s</a><br/>" % (key.generate_url(expires_in=1440, query_auth=True), key.name)
  displayHTML(html_output)

In [24]:
output_links_to_download_sql("select * from test_download_table", 1, ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME, "test-sql-download")